<p style="color:red;font-size:32px;text-align:center"> <b>Social Network Graph Link Prediction  : Part 2</b> </p>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
from pandas import HDFStore
import math
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import gc
from scipy.sparse.linalg import svds as sparse_svd

# 5. Featurizing the graph_data

* Now that we have the graph data with let's build peform  feature engineering and build some features which might be useful in prediction.
* We'll use the train_pos_graph as it's the only graph in train which has valid edges.
* we'll use the same train_pos_graph for building features on test data.

__LOADING THE TRAIN_GRAPH WITH VALID EDGES/LINKS(1)__

In [2]:
train_pos_graph_file = 'processed/after_eda/train_pos.csv'
train_pos_graph = nx.read_edgelist(train_pos_graph_file, delimiter=',', create_using=nx.DiGraph(), nodetype=int)
print("train_graph with edges/links")
print(nx.info(train_pos_graph))

train_graph with edges/links
Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


## 5.1 Similarity Measures

* If we can somehow measure the __degree of proximity between two users__, we can use that measure to predict an edge exists between two nodes.
    
<img src='https://introprogramming.info/wp-content/uploads/2013/07/clip_image026_thumb.png'>
    
* __<font color='red'> NOTE : Graph Theory Terminology </font>__
    - __followers --> <font color='green'> predecessors : nodes directed towards a node </font>__
    - __followees  --> <font color='green'> successors : nodes directed from node </font>__
    - __{19, 1, 7} are the predecessors of {21}__
    - __{31, 14} are the successors of {21}__
       
> * Pic Credits : https://introprogramming.info/tag/graph-implementation/

### 5.1.1 Jaccard Index

* In short, __Jaccard Index__$(J)$ measures the degree of similarity between user_1 and user_2 given that user_1 and user_2 have their features in form of sets.<br>
$$ J = \frac{|user_1 \cap user_2|}{|user_1 \cup user_2|} $$

* Larger value of $(J)$ indicates a higher chance of user_1 and user_2 being connected/having an edge.

* The Jaccard index$(J)$ can be measured for both followers and followees of a user.

In [3]:
def jaccard_index_for_followees(a, b, graph=train_pos_graph):
    
    try:
        x, y = set(graph.successors(a)), set(graph.successors(b))
        
        if len(x) == 0 or len(y) == 0 or len(x & y) == 0:
            return 0
        else:
            return round((len(x & y) / len(x | y)), 4)
    except:
        return 0
    
    
def jaccard_index_for_followers(a, b, graph=train_pos_graph):
    
    try:
        x, y = set(graph.predecessors(a)), set(graph.predecessors(b))
        
        if len(x) == 0 or len(y) == 0 or len(x & y) == 0:
            return 0
        else:
            return round((len(x & y) / len(x | y)), 4)
    except:
        return 0

In [4]:
u1, u2 = 1, 690569
print(f"Jaccard Index for followees between user_{u1} and user_{u2} : {jaccard_index_for_followees(u1, u2)}")
print(f"Jaccard Index for followers between user_{u1} and user_{u2} : {jaccard_index_for_followers(u1, u2)}")

Jaccard Index for followees between user_1 and user_690569 : 0.1053
Jaccard Index for followers between user_1 and user_690569 : 0.0833


In [5]:
u1, u2 = 1545648, 215440
print(f"Jaccard Index for followees between user_{u1} and user_{u2} : {jaccard_index_for_followees(u1, u2)}")
print(f"Jaccard Index for followers between user_{u1} and user_{u2} : {jaccard_index_for_followers(u1, u2)}")

Jaccard Index for followees between user_1545648 and user_215440 : 0
Jaccard Index for followers between user_1545648 and user_215440 : 0


### 5.1.2 Cosine Similarity between sets (Otsuka-Ochiai coefficient)

* This is an other way to measure similarity between two sets of user_1 and user_2 respectively.
<br> $$ C = \frac{|user_1 \cap user_2|}{\sqrt{|user_1| \cdot |user_2|}} $$

* Cosine Similarity$(C)$ is high if the sets of u1 and u2 have high overlap.


In [6]:
def cosine_sim_for_followees(a, b, graph=train_pos_graph):
    
    try:
        x, y = set(graph.successors(a)), set(graph.successors(b))
        
        if len(x) == 0 or len(y) == 0 or len(x & y) == 0:
            return 0
        else:
            return round((len(x & y) / math.sqrt(len(x)* len(y))), 4)
    except:
        return 0
    
    
def cosine_sim_for_followers(a, b, graph=train_pos_graph):
    
    try:
        x, y = set(graph.predecessors(a)), set(graph.predecessors(b))
        
        if len(x) == 0 or len(y) == 0 or len(x & y) == 0:
            return 0
        else:
            return round((len(x & y) / math.sqrt(len(x)* len(y))), 4)
    except:
        return 0

In [7]:
u1, u2 = 1, 690569
print(f"Cosine Sim for followees between user_{u1} and user_{u2} : {cosine_sim_for_followees(u1, u2)}")
print(f"Cosine Sim for followers between user_{u1} and user_{u2} : {cosine_sim_for_followers(u1, u2)}")

Cosine Sim for followees between user_1 and user_690569 : 0.3244
Cosine Sim for followers between user_1 and user_690569 : 0.2408


In [8]:
u1, u2 = 1545648, 215440
print(f"Cosine Sim for followees between user_{u1} and user_{u2} : {cosine_sim_for_followees(u1, u2)}")
print(f"Cosine Sim for followers between user_{u1} and user_{u2} : {cosine_sim_for_followers(u1, u2)}")

Cosine Sim for followees between user_1545648 and user_215440 : 0
Cosine Sim for followers between user_1545648 and user_215440 : 0


## 5.2 Rank Measures

* Given a user, he/she would've have any number of followers/ followees.

* Imagine a user_imp having very large number of followers then that user_imp might have some signifance,<br>
the probability that any random guy on the internet follows user_imp will be very high.

* If any user_j is followed by user_imp then the probability that a random guy follows user_j will also be high.

* We want to capture the significance/importance of each user/node. Page rank algorithm is way to achieve this.

### 5.2.1 Page Rank

* Given a directed graph, the Page Rank algorithm returns a score for each node in the graph<br>
which represents the importance of that node.

* The key point is if a node has many incoming directed edges, then it must be an important one.


<img src='https://i.imgur.com/cPXf8Mv.jpg'>

* Page_C has higher score than Page_E, even though it has more links than Page_C, the link from Page_B which itself has a higher score is directed towards Page_C increases the score of Page_C.

* The idea is a web surfer starting on a random page has 85% likelihood of choosing a random link from that page and 15% likelihood of jumping to a page chosen at random, then  that probability that he will end up on Page_B is 0.384(38.4%).

* The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%. Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

* [NetwrokX Page Rank](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)

> * Pic Credits : [Wiki](https://en.wikipedia.org/wiki/PageRank)

In [9]:
file_path = 'processed/features/page_rank.pkl'

if not os.path.exists(file_path):
    page_rank = nx.pagerank(G=train_pos_graph, alpha=0.85)
    
    with open(file_path, 'wb') as f:
        pickle.dump(page_rank, f)
    print(f'Saved file at : {file_path}')
        
else:
    with open(file_path, 'rb') as f:
        page_rank = pickle.load(f)
    print(f"Loaded file from : {file_path}")

Loaded file from : processed/features/page_rank.pkl


## 5.3 Graph based features

### 5.3.1 Shortest Path

* If the length of the shortest_path between any user_i and user_j is small, then there is a higher chance of user_i and user_j having an edge.

* If user_i and user_j have an edge, then we'll remove that edge and compute the shortest path length.

* If user_i and user_j have no path between them, the shortest path length is -1

* There exists a higher chance of user_1 and user_3 having an edge if
    - user_1 ---> user_2 ---> user_3

In [10]:
def compute_shortest_path_len(a, b, graph=train_pos_graph):
    
    #we'll try to compute the spl(a, b) if a has path to b else return -1
    try:
        #remove the edge if a, b have an edge
        if graph.has_edge(a, b):
            graph.remove_edge(a, b)
            path_len = nx.shortest_path_length(train_pos_graph, source=a, target=b)
            graph.add_edge(a, b)
        #compute the spl if exists
        else:
            path_len = nx.shortest_path_length(train_pos_graph, source=a, target=b)
            
        return path_len
    
    except:
        return -1
        

In [11]:
u1, u2 = 77697, 826021
print(f"length of shortest_path between user_{u1} and user_{u2} : {compute_shortest_path_len(u1, u2)}")

length of shortest_path between user_77697 and user_826021 : 10


In [12]:
u1, u2 = 64454, 964841
print(f"length of shortest_path between user_{u1} and user_{u2} : {compute_shortest_path_len(u1, u2)}")

length of shortest_path between user_64454 and user_964841 : -1


* __<font color='green'> We'll generate a new feature using shortest_path_length. </font>__
* __<font color='red'> spl_coef = 1/(shortest_path_length)  </red>__
* __<font color='green'> This makes sense because node_pairs with less shortest_path_length should've higher weight.</font>__

In [13]:
def spl_coef(a, b):
    
    try:
        #add +1 to make path_length of  node_pairs with no path = 0
        return round(1/(compute_shortest_path_len(a, b) + 1), 4)
    except:
        return 0

In [14]:
u1, u2 = 77697, 826021
print(f"spl_coef between user_{u1} and user_{u2} : {spl_coef(u1, u2)}")

spl_coef between user_77697 and user_826021 : 0.0909


In [15]:
u1, u2 = 64454, 964841
print(f"spl_coef between user_{u1} and user_{u2} : {spl_coef(u1, u2)}")

spl_coef between user_64454 and user_964841 : 0


### 5.3.2 Connectivity of Nodes

__STRONGLY CONNECTED COMPONENTS(SCC)__

* In a Di-graph, Strongly Connected Component refers to a sub_graph which contains a subset of nodes from the graph<br>
such that there exists a path from every node to every other node in the sub_graph.

<img src='https://upload.wikimedia.org/wikipedia/commons/5/5c/Scc.png'>

* The above graph in the above figure contains three SCC sub_graphs.

* If user_i and user_j belong to the same SCC, they may have high degree of similarity,
<br> the SCC can be interpreted as a group which is closely related like friends at work or at college or in a family.

> * Pic Credits : [Wiki](https://en.wikipedia.org/wiki/Strongly_connected_component)

In [16]:
scc = list(nx.strongly_connected_components(train_pos_graph))
print(f"Number of SCCs in train_pos_graph          : {len(scc)}")
print(f"Size of largest SCC in the train_pos_graph : {len(max(scc, key=lambda x : len(x)))} nodes")

Number of SCCs in train_pos_graph          : 610523
Size of largest SCC in the train_pos_graph : 1084279 nodes


In [17]:
#dict mapping nodes to components index
scc_dict = {node : idx for idx, components in enumerate(scc) for node in components}
scc_dict['scc_list'] = scc

In [18]:
def belong_to_same_scc(a, b, scc_dict=scc_dict, graph=train_pos_graph):
    
    try:
        #if a --> and b --> a, then it's a SCC,
        if graph.has_edge(a, b) and graph.has_edge(b, a):
            return 1
    
        #if path exists from a to b, and (a, b) belong to the same scc
        elif (compute_shortest_path_len(a, b) != -1) and \
              b in scc_dict.get('scc_list')[scc_dict.get(a)]:
            return 1
        
        else:
            return 0
    except:
        return 0

In [19]:
u1, u2 = 1723, 1725 #same scc
print(f"Does user_{u1} and user_{u2} belong to the same SCC ? : {bool(belong_to_same_scc(u1, u2))}")

Does user_1723 and user_1725 belong to the same SCC ? : True


In [20]:
u1, u2 = 1723, 40, #not from same scc
print(f"Does user_{u1} and user_{u2} belong to the same SCC ? : {bool(belong_to_same_scc(u1, u2))}")

Does user_1723 and user_40 belong to the same SCC ? : False


In [21]:
#u1 and u189226 are not parallel edges, but belong to same scc
u1, u2 = 1, 189226
print(f"Does user_{u1} and user_{u2} belong to the same SCC ? : {bool(belong_to_same_scc(u1, u2))}")

Does user_1 and user_189226 belong to the same SCC ? : True


__WEEKLY CONNECTED COMPONENTS(SCC)__

* In a Di-Graph with directions ignored, Weekly Connected Component refers to a sub_graph,
<br>where there exists a path from every node to every other node in the sub_graph.

* The entire graph in the above figure from  SCC section is a WCC is all the directions are ignored.
<br> There always exists a path from any node to any other node.

<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSW8QbL3VMYirQBNIRQWxM-7RaFR_F70prymhBkU7IWIrTOd6TI'>

* The WCC can be interpreted as a community where users have some similarity like culture, ethinicity, country etc,.

> * Pic Credits : https://www.quora.com/What-are-strongly-and-weakly-connected-components

In [22]:
wcc = list(nx.weakly_connected_components(train_pos_graph))
print(f"Number of WCCs in train_pos_graph          : {len(wcc)}")
print(f"Size of largest WCC in the train_pos_graph : {len(max(wcc, key=lambda x : len(x)))} nodes")

Number of WCCs in train_pos_graph          : 48602
Size of largest WCC in the train_pos_graph : 1650233 nodes


In [23]:
#dict mapping nodes to scc_components index
wcc_dict = {node : idx for idx, components in enumerate(wcc) for node in components}
wcc_dict['wcc_list'] = wcc

In [24]:
def belong_to_same_wcc(a, b, wcc_dict=wcc_dict, graph=train_pos_graph):
    
    try:
        # if b --> a, then there is an edge return 1
        if graph.has_edge(b, a):
            return 1
    
        #return 1  path exists from a --> b after  and b --> a
        #see compute_shortest_path_len() for more
        elif compute_shortest_path_len(a, b, graph=train_pos_graph) != -1 and\
             compute_shortest_path_len(b, a, graph=train_pos_graph) != -1:
            return 1
    
        if b in wcc_dict.get('wcc_list')[wcc_dict.get(a)]:
            return 1
        
        else:
            return 0
    except:
        return 0

In [25]:
u1, u2 = 1723, 1725 #belong to same wcc
print(f"user_{u1} and user_{u2} belong to the same WCC ? : {bool(belong_to_same_wcc(u1, u2))}")

user_1723 and user_1725 belong to the same WCC ? : True


In [26]:
u1, u2 = 1723, 433241
print(f"user_{u1} and user_{u2} belong to the same WCC ? : {bool(belong_to_same_wcc(u1, u2))}")

user_1723 and user_433241 belong to the same WCC ? : False


In [27]:
#u 433241 and u1228670 aren't neighbors but they belong to same wcc
u1, u2 = 433241, 1228670
print(f"user_{u1} and user_{u2} belong to the same WCC ? : {bool(belong_to_same_wcc(u1, u2))}")

user_433241 and user_1228670 belong to the same WCC ? : True


In [28]:
u1, u2 = 35, 1228670
print(f"user_{u1} and user_{u2} belong to the same WCC ? : {bool(belong_to_same_wcc(u1, u2))}")

user_35 and user_1228670 belong to the same WCC ? : False


### 5.3.3 Adamic/Adar Index

* The concept of Adar Index $(A)$ is based on neighbours of user_i and user_j.
$$A(u_i ,u_j)=\sum_{u \in N(u_i) \cap N(u_j)}\frac{1}{log(|N(u)|)}$$


* Adar Index $(A)$ measure the degree of user_i and user_j being friends using their common neighbor's neighborhood.
    - if $(A)$ is high, user_i and user_j are likely to be friends and vice versa.


* Assume user_i and user_j are linked to some set of users, then the likelihood of user_i and user_j being friends is
    - is small if  the neighborhood of overlap between the successors(user_i) and the successors(user_j) is large.
    - is large if the neighborhood of overlap between the successors(user_i) and the successors(user_j) is small.


* [Adar Index Wiki](https://en.wikipedia.org/wiki/Adamic/Adar_index)

In [29]:
def adar_index(a, b, graph=train_pos_graph):
    
    try:
        #find common set of users
        common_users = set(train_pos_graph.successors(a)) & set(train_pos_graph.successors(b))
        if len(common_users) != 0:
            #small if common_users have large number of predecessors/followers, 
            #ie,. if the common set of users are celebrities, then they'll ve large number of followers.
            return round(sum([1/math.log10(len(list(train_pos_graph.predecessors(c)))) for c in common_users]), 4)
        else:
            return 0
    except:
        return 0
          

In [30]:
u1, u2 = 1, 690569
print(f"Adar Index of user_{u1} and user_{u2} : {adar_index(u1, u2)}")

Adar Index of user_1 and user_690569 : 2.8303


In [31]:
u1, u2 = 657672, 1094233 #these user have high follower count
print(f"Adar Index of user_{u1} and user_{u2} : {adar_index(u1, u2)}")

Adar Index of user_657672 and user_1094233 : 16.8503


In [32]:
u1, u2 = 1, 189226 
print(f"Adar Index of user_{u1} and user_{u2} : {adar_index(u1, u2)}")

Adar Index of user_1 and user_189226 : 0


### 5.3.4 follow_back/parallel_edge

* We can see in EDA : parellel edges, that __6.4M of 9.46M edges__ are simply follow backs back and forth.


* The chance of user_i being followed back by a user_j is high if user_i follows user_j.
    - if user_i --> user_j, the chance of user_j --> user_i is high.

In [33]:
def follows_back(a, b, graph=train_pos_graph):
    
    try:
        return int(graph.has_edge(b, a))
    except:
        return 0

In [34]:
u1, u2 = 1, 690569
print(f"user_{u2} follows user_{u1} ? : {bool(follows_back(u1, u2))}")

user_690569 follows user_1 ? : True


In [35]:
u1, u2 = 657672, 1094233 
print(f"user_{u2} follows user_{u1} ? : {bool(follows_back(u1, u2))}")

user_1094233 follows user_657672 ? : True


In [36]:
u1, u2 = 2, 1
print(f"user_{u2} follows user_{u1} ? : {bool(follows_back(u1, u2))}")

user_1 follows user_2 ? : False


### 5.3.5 Katz Centrality

* Similar to Page Rank, measures the influence/importance of a node in the graph.

* [Katz Centrality Wiki](https://en.wikipedia.org/wiki/Katz_centrality)

* [Katz Centrality GeeksForGeeks](https://www.geeksforgeeks.org/katz-centrality-centrality-measure/)

In [37]:
file_path = 'processed/features/katz_cen.pkl'

if not os.path.exists(file_path):
    katz = nx.katz.katz_centrality(train_pos_graph, alpha=0.005, beta=1)
    with open(file_path, 'wb') as f:
        pickle.dump(katz, f)
    print(f'File saved at : {file_path}')
else:
    print(f'Loading File from : {file_path}')
    with open(file_path, 'rb') as f:
        katz = pickle.load(f)

Loading File from : processed/features/katz_cen.pkl


In [38]:
print(f"min katz score  : {katz[min(katz, key=katz.get)] : .6f}")
print(f"max katz score  : {katz[max(katz, key=katz.get)] : .6f}")
print(f"mean katz score : {sum(katz.values())/len(katz) : .6f}")

min katz score  :  0.000731
max katz score  :  0.003308
mean katz score :  0.000748


### 5.3.6 HITS Score

* The HITS algorithm computes two numbers for a node. 
    - Authorities estimates the node value based on the incoming links. 
    - Hubs estimates the node value based on outgoing links.


* [Hits Score Wiki](https://en.wikipedia.org/wiki/HITS_algorithm)

In [39]:
file_path = 'processed/features/hits_score.pkl'

if not os.path.exists(file_path):
    hits_score = nx.hits(train_pos_graph)
    with open(file_path, 'wb') as f:
        pickle.dump(hits_score, f)
    print(f'File saved at : {file_path}')
else:
    print(f'Loading File from : {file_path}')
    with open(file_path, 'rb') as f:
        hits_score = pickle.load(f)

Loading File from : processed/features/hits_score.pkl


In [40]:
print(f"min hits score  : {hits_score[0][min(hits_score[0], key=hits_score[0].get)] : .6f}")
print(f"max hits score  : {hits_score[0][max(hits_score[0], key=hits_score[0].get)] : .6f}")
print(f"mean hits score : {sum(hits_score[0].values())/len(hits_score[0]) : .6f}")

min hits score  :  0.000000
max hits score  :  0.004869
mean hits score :  0.000001


### 5.3.7 Preferential Attachment

* The idea here is that a user with many friends tends to create more connections in future.

* This feature may not make any sense, but this is due to the fact from finance saying the rich gets richer.

* The richness of two users is obatined by multiplying their number of friends/followers.

* $PT = |user_1| \cdot |user_2|$

In [43]:
def preferential_attachment(a, b, graph=train_pos_graph):
    
    try:
        n_a = len(set(graph.neighbors(a)))
        n_b = len(set(graph.neighbors(b)))
        return n_a * n_b
        
    except:
        return 0

# 6. Featurization

## 6.1 Sampling from train and test

* Due to computational limitations, we'll sample a subset of datapoints for both train and test.

In [48]:
train_path = 'processed/final/'
test_path = 'processed/final/'

with open(os.path.join(train_path, 'Ytrain.csv')) as f_train:
    len_train = len(f_train.readlines())
    print(f"Number of node_pairs in train : {len_train}")
    
with open(os.path.join(test_path, 'Ytest.csv')) as f_test:
    len_test = len(f_test.readlines())
    print(f"Number of node_pairs in test  : {len_test}")

Number of node_pairs in train : 15100030
Number of node_pairs in test  : 3775008


In [49]:
train_sample_size = 100_000
test_sample_size = 50_000
col_names = ['source', 'destination']

#effiecient than df.sample(sample_size)
skip_train_inidices = np.random.choice(np.arange(len_train), size=len_train - train_sample_size, replace=False)
skip_test_inidices = np.random.choice(np.arange(len_test), size=len_test - test_sample_size, replace=False)

skip_train_inidices.sort()
skip_test_inidices.sort()

__SAMPLING FROM TRAIN__

In [50]:
train_df = pd.read_csv(os.path.join(train_path, 'Xtrain.csv'), skiprows=skip_train_inidices, names=col_names)
train_df['edge_indicator'] = pd.read_csv(os.path.join(train_path, 'Ytrain.csv'), skiprows=skip_train_inidices, names=['edge_indicator'])
train_df.sample(5)

,source,destination,edge_indicator
10962,21242,335712,1
67417,1506177,1214763,0
89550,790393,1637176,0
92816,1525423,1395321,0
50447,750747,517082,0


In [51]:
print(f"shape of train_df            : {train_df.shape}")
print(f"num of node_pairs with edges : {(train_df.edge_indicator == 1).sum()}")
print(f"num of node_pairs with edges : {(train_df.edge_indicator == 0).sum()}")

shape of train_df            : (100000, 3)
num of node_pairs with edges : 49840
num of node_pairs with edges : 50160


__SAMPLING FROM TEST__

In [52]:
test_df = pd.read_csv(os.path.join(test_path, 'Xtest.csv'), skiprows=skip_test_inidices, names=col_names)
test_df['edge_indicator'] = pd.read_csv(os.path.join(test_path, 'Ytest.csv'), skiprows=skip_test_inidices, names=['edge_indicator'])
test_df.sample(5)

,source,destination,edge_indicator
15455,245864,383916,1
19248,405992,317798,1
5062,35807,1237806,1
32426,1797929,994643,0
4687,682440,1515360,1


In [53]:
print(f"shape of train_df            : {test_df.shape}")
print(f"num of node_pairs with edges : {(test_df.edge_indicator == 1).sum()}")
print(f"num of node_pairs with edges : {(test_df.edge_indicator == 0).sum()}")

shape of train_df            : (50000, 3)
num of node_pairs with edges : 24984
num of node_pairs with edges : 25016


## 6.2 Adding set of features

* __We will create  each of these 10 features for both train and test.__
    - jaccard_followers
    - jaccard_followees
    - cosine_followers
    - cosine_followees
    - follows_back
    - belong_to_same_scc
    - belong_to_same_wcc
    - adar_index
    - prefer_attach
    - spl_coef

In [56]:
def build_data_matrix_stage1(df):

    df['jaccard_followers'] = df.apply(lambda row : jaccard_index_for_followers(row['source'], row['destination']), axis=1)
    df['jaccard_followees'] = df.apply(lambda row : jaccard_index_for_followees(row['source'], row['destination']), axis=1)
    df['cosine_followers'] = df.apply(lambda row : cosine_sim_for_followers(row['source'], row['destination']), axis=1)
    df['cosine_followees'] = df.apply(lambda row : cosine_sim_for_followees(row['source'], row['destination']), axis=1)
    df['follows_back'] = df.apply(lambda row : follows_back(row['source'], row['destination']), axis=1)
    df['same_scc'] = df.apply(lambda row : belong_to_same_scc(row['source'], row['destination']), axis=1)
    df['same_wcc'] = df.apply(lambda row : belong_to_same_wcc(row['source'], row['destination']), axis=1)
    df['adar_index'] = df.apply(lambda row : adar_index(row['source'], row['destination']), axis=1)
    df['spl_coef'] = df.apply(lambda row : spl_coef(row['source'], row['destination']), axis=1)
    df['prefer_attach'] = df.apply(lambda row : preferential_attachment(row['source'], row['destination']), axis=1)
    
    return df

In [57]:
%%time

train_df = build_data_matrix_stage1(train_df)
test_df = build_data_matrix_stage1(test_df)

Wall time: 7min 10s


In [58]:
print(f"Number of features after stage1 : {train_df.shape[1] - 3}")

Number of features after stage1 : 10


In [59]:
test_df.sample(5)

,source,destination,edge_indicator,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,follows_back,same_scc,same_wcc,adar_index,spl_coef,prefer_attach
17539,1800078,1067820,1,0.2182,0.2449,0.3814,0.3961,1,1,1,10.5572,0.3333,918
44260,348622,887334,0,0.0000,0.0000,0.0000,0.0000,0,0,1,0.0000,0.0000,0
38481,1285137,329686,0,0.0000,0.0000,0.0000,0.0000,0,1,1,0.0000,0.1111,9
17763,1823666,1694208,1,0.0000,0.0000,0.0000,0.0000,0,1,1,0.0000,0.2500,12
31133,1647420,852350,0,0.0000,0.0000,0.0000,0.0000,0,0,1,0.0000,0.0000,0


__STAGE 2__

* __Adding 6 new features__
    - num_followers_source
    - num_followees_source
    - num_followers_dest
    - num_followees_dest
    - inter_followers
    - inter_followees


In [61]:
def build_data_matrix_stage2(df, graph=train_pos_graph):
    
    num_followers_source = []
    num_followees_source = []
    num_followers_dest = []
    num_followees_dest = []
    inter_followers = []
    inter_followees = []
    
    for index, row in df.iterrows():
        
        try:
            s1=set(graph.predecessors(row['source']))
            s2=set(graph.successors(row['source']))
        except:
            s1 = set()
            s2 = set()
            
        try:
            d1=set(graph.predecessors(row['destination']))
            d2=set(graph.successors(row['destination']))
        except:
            d1 = set()
            d2 = set()
            
        num_followers_source.append(len(s1))
        num_followees_source.append(len(s2))

        num_followers_dest.append(len(d1))
        num_followees_dest.append(len(d2))

        inter_followers.append(len(s1 & d1))
        inter_followees.append(len(s2 & d2))
        
    df['num_followers_source'] = num_followers_source
    df['num_followers_dest'] = num_followers_dest
    df['num_followees_source'] = num_followees_source
    df['num_followees_dest'] = num_followees_dest
    df['inter_followers'] = inter_followers
    df['inter_followees'] = inter_followees
    
    return df

In [62]:
%%time

train_df = build_data_matrix_stage2(train_df)
test_df = build_data_matrix_stage2(test_df)

Wall time: 10.9 s


In [63]:
print(f"Number of features after stage2 : {train_df.shape[1] - 3}")

Number of features after stage2 : 16


In [64]:
test_df.sample(5)

,source,destination,edge_indicator,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,follows_back,same_scc,same_wcc,adar_index,spl_coef,prefer_attach,num_followers_source,num_followers_dest,num_followees_source,num_followees_dest,inter_followers,inter_followees
26847,576747,704661,0,0.0,0.0000,0.0,0.000,0,0,1,0.0000,0.1000,1,0,1,1,1,0,0
30718,123637,1084081,0,0.0,0.0000,0.0,0.000,0,0,0,0.0000,0.0000,1,0,1,1,1,0,0
23352,1207155,216377,1,0.0,0.0000,0.0,0.000,0,1,1,0.0000,0.2000,588,48,3,196,3,0,0
5925,1714822,800837,1,0.0,0.2667,0.0,0.434,0,1,1,9.4023,0.3333,2124,22,158,59,36,0,20
36867,760362,1648060,0,0.0,0.0000,0.0,0.000,0,1,1,0.0000,0.1111,8,2,2,4,2,0,0


## 6.3 Adding new set of features

* __We'll create each of these 14 features below for both train and test.__


* Weight Features
    - weight of incoming edges
    - weight of outgoing edges
    - weight of incoming edges + weight of outgoing edges
    - weight of incoming edges * weight of outgoing edges
    - 2*weight of incoming edges + weight of outgoing edges
    - weight of incoming edges + 2*weight of outgoing edges
    
    
* Page_Ranking of source
* Page_Ranking of destination
* Katz score of source
* Katz score of destination
* Hubs score of source
* Hubs score of destination
* Authorities score of source
* Authorities score of destination

__WEIGHT FEATURES__

* Inorder to determinine the similarity of nodes, an edge weight value is calculated between the nodes.
<br>Edge weight decreases as the neighbor count increases.

* Intuitively, Consider 1M people following a celebrity on a social network then the chances
<br> of most them meeting each other or the celebrity is small.

* On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other.

* $$\begin{equation} W = \frac{1}{\sqrt{1+|X|}} \end{equation} $$
<br>$\hspace{11cm}| X | = \text{length of the node}$

> * Credits : Graph-based Features for Supervised Link Prediction , William Cukierski, Benjamin Hamner, Bo Yang

In [65]:
weights_incoming = {}
weights_outgoing = {}

for node in train_pos_graph.nodes():
    weights_incoming[node] = 1 / math.sqrt(1 + len(set(train_pos_graph.predecessors(node))))
    weights_outgoing[node] = 1 / math.sqrt(1 + len(set(train_pos_graph.successors(node))))
    
#mean values for imputation
mean_weight_in = np.mean(list(weights_incoming.values()))
mean_weight_out = np.mean(list(weights_outgoing.values()))

__STAGE 3__

In [66]:
def build_data_matrix_stage3(df,\
                             weights_in=weights_incoming,\
                             weights_out=weights_outgoing,\
                             page_rank_dict=page_rank,\
                             katz_dict=katz,\
                             hits_dicts_list=hits_score):
    
    mean_w_in = np.mean(list(weights_incoming.values()))
    mean_w_out = np.mean(list(weights_outgoing.values()))
    
    df['weight_in'] = df.destination.apply(lambda x : weights_in.get(x, mean_w_in))
    df['weight_out'] = df.source.apply(lambda x : weights_out.get(x, mean_w_out))
    
    df['weight_f1'] = df.weight_in * df.weight_out
    df['weight_f2'] = df.weight_in + df.weight_out
    df['weight_f3'] = 2*df.weight_in + df.weight_out
    df['weight_f4'] = df.weight_in + 2*df.weight_out
    
    mean_page_rank = np.mean(list(page_rank_dict.values()))
    df['page_rank_source'] = df.source.apply(lambda x : page_rank_dict.get(x, mean_page_rank))
    df['page_rank_dest'] = df.destination.apply(lambda x : page_rank_dict.get(x, mean_page_rank))
    
    mean_katz = np.mean(list(katz_dict.values()))
    df['katz_source'] = df.source.apply(lambda x : katz_dict.get(x, mean_katz))
    df['katz_dest'] = df.destination.apply(lambda x : katz_dict.get(x, mean_katz))
    
    df['hub_source'] = df.source.apply(lambda x : hits_dicts_list[0].get(x, 0))
    df['hub_dest'] = df.destination.apply(lambda x : hits_dicts_list[0].get(x, 0))
    
    df['auth_source'] = df.source.apply(lambda x : hits_dicts_list[1].get(x, 0))
    df['auth_dest'] = df.destination.apply(lambda x : hits_dicts_list[1].get(x, 0))
    
    return df

In [67]:
%%time

train_df = build_data_matrix_stage3(train_df)
test_df = build_data_matrix_stage3(test_df)

Wall time: 2.54 s


In [68]:
print(f"Number of features after stage3 : {train_df.shape[1] - 3}")

Number of features after stage3 : 30


In [69]:
test_df.sample(5)

,source,destination,edge_indicator,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,follows_back,same_scc,same_wcc,...,weight_f3,weight_f4,page_rank_source,page_rank_dest,katz_source,katz_dest,hub_source,hub_dest,auth_source,auth_dest
39049,628174,813764,0,0.0,0.0000,0.0,0.0000,0,0,1,...,2.447214,1.894427,2.445197e-07,2.253640e-07,0.000744,0.000735,3.292557e-17,5.561376e-18,6.820513e-18,0.000000e+00
27601,1736000,118976,0,0.0,0.0000,0.0,0.0000,0,0,1,...,2.500000,2.000000,4.839489e-07,5.616324e-07,0.000739,0.000748,2.055789e-20,7.660612e-21,1.061153e-19,0.000000e+00
14747,1342990,998345,1,0.0,0.0000,0.0,0.0000,0,1,1,...,0.469368,0.431644,3.991159e-06,1.704890e-06,0.000907,0.000853,8.543959e-15,3.876832e-13,8.695359e-15,1.882114e-14
7943,997401,875481,1,0.0,0.0000,0.0,0.0000,1,1,1,...,1.207107,1.353553,4.552738e-07,8.459678e-07,0.000750,0.000754,6.279346e-18,3.165929e-17,3.569346e-16,3.694076e-16
12769,103174,694319,1,0.0,0.1111,0.0,0.2041,1,1,1,...,1.272392,1.203143,4.727746e-07,2.895682e-07,0.000760,0.000754,5.097587e-16,3.184172e-16,1.170909e-14,4.575561e-17


## 6.4 SVD Features

* Our graph can be represented as an __Adjacency matrix__.
    - It's a square shape : (num_nodes * num_nodes)
    - An entry of 1 at $A_{ij}$ in the matrix represents the presence of an edge between node_i and node_j.
    - 0 represents the absence of an edge.
    
<img src='http://mathworld.wolfram.com/images/eps-gif/AdjacencyMatrix_1002.gif'>

* The above figure represents adjacency matrices for undirected graphs.

* We'll decompose the adjacency matrix into (U, S, V) using SVD.

* We'll use the top 6 eigen vectors representing the node from U and V.


* __A total of24 features will be generated from SVD__
    - U_source(6features),
    - U_dest(6features),
    - V_source(6features),
    - V_dest(6features)


> * Pic Credits : [wolfram](http://mathworld.wolfram.com/AdjacencyMatrix.html)

In [70]:
#dict mapping indices to nodes
adjacency_dict = {node : idx for idx, node in enumerate(sorted(train_pos_graph.nodes()))}

In [71]:
adjacency_matrix = nx.adjacency_matrix(train_pos_graph, nodelist=list(adjacency_dict.values())).asfptype()
print(f"Shape of adjacency matrix : {adjacency_matrix.get_shape()}")
print(f"Type of adjacency matrix : {type(adjacency_matrix)}")

#you don't need this from now
del train_pos_graph
gc.collect()

Shape of adjacency matrix : (1780722, 1780722)
Type of adjacency matrix : <class 'scipy.sparse.csr.csr_matrix'>


0

In [72]:
U, S, V = sparse_svd(adjacency_matrix, k=6)

print(f"shape of left singular matrix (U)  : {U.shape}")
print(f"shape of diagonal matrix (S)       : {S.shape}")
print(f"shape of right singular matrix (U) : {V.shape}")

shape of left singular matrix (U)  : (1780722, 6)
shape of diagonal matrix (S)       : (6,)
shape of right singular matrix (U) : (6, 1780722)


In [98]:
def get_svd_vals(node, M, adj_dict=adjacency_dict):
    
    try:
        return M[adj_dict[node]]
    except:
        return np.zeros(6)

__STAGE 4__

In [104]:
def build_data_matrix_stage4(df, U=U, V=V):
    
    df[['svd_us1', 'svd_us2', 'svd_us3', 'svd_us4', 'svd_us5', 'svd_us6']] = \
                                    df.source.apply(lambda x : get_svd_vals(x, U)).apply(pd.Series)
    
    df[['svd_ud1', 'svd_ud2', 'svd_ud3', 'svd_ud4', 'svd_ud5', 'svd_ud6']] = \
                                    df.destination.apply(lambda x : get_svd_vals(x, U)).apply(pd.Series)
    
    df[['svd_vs1', 'svd_vs2', 'svd_vs3', 'svd_vs4', 'svd_vs5', 'svd_vs6']] = \
                                    df.source.apply(lambda x : get_svd_vals(x, V.T)).apply(pd.Series)
    
    df[['svd_vd1', 'svd_vd2', 'svd_vd3', 'svd_vd4', 'svd_vd5', 'svd_vd6']] = \
                                    df.destination.apply(lambda x : get_svd_vals(x, V.T)).apply(pd.Series)
    
    df['svd_dot_u'] = df.apply(lambda row : np.dot(get_svd_vals(row['source'], U), get_svd_vals(row['destination'], U)), axis=1)
    df['svd_dot_v'] = df.apply(lambda row : np.dot(get_svd_vals(row['source'], V.T), get_svd_vals(row['destination'], V.T)), axis=1)
                               

    return df

In [105]:
%%time

train_df = build_data_matrix_stage4(train_df)
test_df = build_data_matrix_stage4(test_df)

Wall time: 2min 4s


In [106]:
print(f"Number of features after stage4 : {train_df.shape[1] - 3}")

Number of features after stage4 : 56


In [107]:
test_df.sample(5)

,source,destination,edge_indicator,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,follows_back,same_scc,same_wcc,...,svd_vs5,svd_vs6,svd_vd1,svd_vd2,svd_vd3,svd_vd4,svd_vd5,svd_vd6,svd_dot_u,svd_dot_v
41059,1463415,1826881,0,0.0,0.0,0.0,0.0,0,1,1,...,1.202284e-12,-9.537461e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.386193e-27,0.000000e+00
39786,1082079,385065,0,0.0,0.0,0.0,0.0,0,0,1,...,0.000000e+00,0.000000e+00,-2.648636e-14,-1.327009e-13,3.049329e-07,-1.056872e-14,2.291168e-13,-7.397369e-14,1.854848e-19,0.000000e+00
2444,1679549,360816,1,0.0,0.0,0.0,0.0,1,0,1,...,0.000000e+00,0.000000e+00,-7.690260e-15,-9.489635e-15,1.447888e-09,-3.285758e-15,1.257927e-13,-5.370567e-16,-3.611753e-30,0.000000e+00
12816,72077,1521464,1,1.0,0.0,1.0,0.0,0,1,1,...,8.795732e-15,-7.238315e-16,-1.591296e-16,-1.309760e-15,2.118414e-14,-2.412886e-16,2.341119e-15,-1.423428e-18,-3.458982e-29,8.504702e-26
25126,449054,91403,0,0.0,0.0,0.0,0.0,0,0,1,...,9.144413e-16,-2.136290e-18,-2.519816e-14,-4.519128e-13,1.611155e-12,-1.067757e-13,1.682590e-15,-8.164675e-16,9.104525e-25,3.688883e-26


In [109]:
file_path = 'processed/features/features_train_test.h5'

if not os.path.exists(file_path):
    hdf_table = HDFStore(file_path)
    hdf_table.put('train_df', train_df, format='table', data_columns=True)
    hdf_table.put('test_df', test_df, format='table', data_columns=True)
    hdf_table.close()
    print(f'Saved file at : {file_path}')
    
else:
    print(f'File already exists at : {file_path}')

Saved file at : processed/features/features_train_test.h5


__TOTAL FEATURES BUILT__

In [110]:
train_df.columns

Index(['source', 'destination', 'edge_indicator', 'jaccard_followers',
       'jaccard_followees', 'cosine_followers', 'cosine_followees',
       'follows_back', 'same_scc', 'same_wcc', 'adar_index', 'spl_coef',
       'prefer_attach', 'num_followers_source', 'num_followers_dest',
       'num_followees_source', 'num_followees_dest', 'inter_followers',
       'inter_followees', 'weight_in', 'weight_out', 'weight_f1', 'weight_f2',
       'weight_f3', 'weight_f4', 'page_rank_source', 'page_rank_dest',
       'katz_source', 'katz_dest', 'hub_source', 'hub_dest', 'auth_source',
       'auth_dest', 'svd_us1', 'svd_us2', 'svd_us3', 'svd_us4', 'svd_us5',
       'svd_us6', 'svd_ud1', 'svd_ud2', 'svd_ud3', 'svd_ud4', 'svd_ud5',
       'svd_ud6', 'svd_vs1', 'svd_vs2', 'svd_vs3', 'svd_vs4', 'svd_vs5',
       'svd_vs6', 'svd_vd1', 'svd_vd2', 'svd_vd3', 'svd_vd4', 'svd_vd5',
       'svd_vd6', 'svd_dot_u', 'svd_dot_v'],
      dtype='object')

__<font color='red'> Please Open the Next Notebook : Part 3</font>__